In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import ete3
import glob
import rooting_methods_ete3
# import numpy as np

# To do:
1. Write summary statistic dataframes for each dataset!

# Custom function definitions

In [3]:
def is_monophyletic_ete3(hypothetical_root, previous_root, tree, test_set, monophyletic_list):
    """
    This is a bit of a mess but it works. Wish it were cleaner. Makes successive recursive calls. Clean later.
    """
    if hypothetical_root.is_root() == True:
        l_clade = hypothetical_root.children[0]
        r_clade = hypothetical_root.children[1]
        l_terms = set([i.name[:5] for i in l_clade.get_leaves()])
        r_terms = set([i.name[:5] for i in r_clade.get_leaves()])
        #This does the test. Essentially, make sure all terms on one side are in the test_set
        #and make sure that no terms on the otherside are in the test_set
        if len(test_set.union(l_terms))-len(test_set) == 0 and\
            len(test_set.intersection(r_terms)) == 0:
                monophyletic_list.append((l_clade, r_clade))
                return monophyletic_list
        elif len(test_set.union(r_terms))-len(test_set) == 0 and\
            len(test_set.intersection(l_terms)) == 0:
                monophyletic_list.append((r_clade, l_clade))
                return monophyletic_list
    else:
        #For non-root nodes, we don't care about left and right as much as downstream and upstream
        ds_terms = set([i.name[:5] for i in hypothetical_root.get_leaves()])
        us_terms = set([i.name[:5] for i in tree.get_leaves()]) - ds_terms
        if len(test_set.union(ds_terms))-len(test_set) == 0 and\
            len(test_set.intersection(us_terms)) == 0:
            monophyletic_list.append((hypothetical_root, previous_root))
        if len(test_set.union(us_terms))-len(test_set) == 0 and\
            len(test_set.intersection(ds_terms)) == 0:
            monophyletic_list.append((hypothetical_root, previous_root))
    #Recursion, depth first search
    if len(hypothetical_root.children) == 2:
        l_clade, r_clade = hypothetical_root.children
        monophyletic_list = is_monophyletic_ete3(l_clade, hypothetical_root, tree, test_set, monophyletic_list)
        monophyletic_list = is_monophyletic_ete3(r_clade, hypothetical_root, tree, test_set, monophyletic_list)
    elif len(hypothetical_root.children) == 0:
        return monophyletic_list
    return monophyletic_list

def label_internals(tree):
    """
    Just labeling all internal non-root nodes with a simple pattern to make subsequent life much easier.
    """
    assert len(tree.children) == 2 ###Require bifurcating root
    
    ###Start each side out with zeros
    if not tree.children[0].is_leaf():
        tree.children[0].name = 'IntA_0'
    if not tree.children[0].is_leaf():
        tree.children[1].name = 'IntB_0'
    ###Label successive internals    
    for i, node in enumerate(tree.children[0].get_descendants()):
        if not node.is_leaf():
            node.name = 'IntA_{}'.format(i+1)
    for i, node in enumerate(tree.children[1].get_descendants()):
        if not node.is_leaf():
            node.name = 'IntB_{}'.format(i+1)
    return

# First dataset

In [4]:
direc_to_analyze = '../Data/OMA_orthologs/5204_4890/'
taxa_to_analyze = direc_to_analyze.split('/')[-2].split('_')
print(taxa_to_analyze)

['5204', '4890']


In [5]:
species_df = pd.read_csv('../Data/oma-species-full.tsv', sep='\t')
species_df.head()

,OMA_code,TaxID,Scientific_name,Source,Release,Taxonomy_lineage
0,ACAM1,329726,Acaryochloris marina (strain MBIC 11017),Genome Reviews,"18-MAR-2008 (Rel. 88, Last updated, Version 2)",131567;2;1783272;1798711;1117;1890424;1890429;...
1,ACCPU,522306,Accumulibacter phosphatis (strain UW-1),EBI,"15-MAY-2014 (Rel. 120, Last updated, Version 4)",131567;2;1224;28216;119066;327159;327160
2,ACEAZ,574087,Acetohalobium arabaticum (strain ATCC 49924 / ...,EBI,"09-JAN-2014 (Rel. 119, Last updated, Version 5)",131567;2;1783272;1239;186801;53433;53434;28186...
3,ACEMN,891968,Acetomicrobium mobile (strain ATCC BAA-54 / DS...,EBI,"23-JUL-2013 (Rel. 117, Last updated, Version 3)",131567;2;508458;649775;649776;649777;49894;97477
4,ACEP3,634452,Acetobacter pasteurianus (strain NBRC 3283 / L...,Genome Reviews,"24-NOV-2009 (Rel. 114, Last updated, Version 2)",131567;2;1224;28211;204441;433;434;438;481145


In [6]:
lside_names = set(species_df[species_df['Taxonomy_lineage'].\
                                          str.contains(';{};'.format(taxa_to_analyze[0]))]['OMA_code'])
rside_names = set(species_df[species_df['Taxonomy_lineage'].\
                                          str.contains(';{};'.format(taxa_to_analyze[1]))]['OMA_code'])
print(len(lside_names), len(rside_names))

36 90


In [7]:
found_roots = []
for tree_file in glob.glob(direc_to_analyze + '*.treefile'):
    tree = ete3.Tree(tree_file)
    rooting_methods_ete3.MP_root_ete3(tree)
    listy = is_monophyletic_ete3(tree, '', tree, set(lside_names), [])
#     listy = is_monophyletic_ete3(tree, '', tree, set(rside_names), [])
    found_roots.append(len(listy))
    if len(listy) == 1:
        tree.set_outgroup(listy[0][0])
        assert set(tree.children) == set(listy[0])
        
        if lside_names.union(set([i.name[:5] for i in tree.children[0].get_leaves()])) == lside_names:
            pass
        elif rside_names.union(set([i.name[:5] for i in tree.children[0].get_leaves()])) == rside_names:
            tree.swap_children()
        else:
            print('big error')
        assert lside_names.union(set([i.name[:5] for i in tree.children[0].get_leaves()])) == lside_names
        
        new_file = tree_file.replace('.afa.treefile', '.True.newick')
        label_internals(tree)
        tree.write(format=1, outfile=new_file)        
print('Successes (out of):', len([i for i in found_roots if i == 1]),  len(found_roots))
print('Possible errors:', len([i for i in found_roots if i > 1]))

Successes (out of): 1985 2137
Possible errors: 0


# Second dataset

In [8]:
direc_to_analyze = '../Data/OMA_orthologs/5204_4890_33511_33317/'
taxa_to_analyze = direc_to_analyze.split('/')[-2].split('_')
print(taxa_to_analyze)
species_df = pd.read_csv('../Data/oma-species-full.tsv', sep='\t')
species_df.head()

['5204', '4890', '33511', '33317']


,OMA_code,TaxID,Scientific_name,Source,Release,Taxonomy_lineage
0,ACAM1,329726,Acaryochloris marina (strain MBIC 11017),Genome Reviews,"18-MAR-2008 (Rel. 88, Last updated, Version 2)",131567;2;1783272;1798711;1117;1890424;1890429;...
1,ACCPU,522306,Accumulibacter phosphatis (strain UW-1),EBI,"15-MAY-2014 (Rel. 120, Last updated, Version 4)",131567;2;1224;28216;119066;327159;327160
2,ACEAZ,574087,Acetohalobium arabaticum (strain ATCC 49924 / ...,EBI,"09-JAN-2014 (Rel. 119, Last updated, Version 5)",131567;2;1783272;1239;186801;53433;53434;28186...
3,ACEMN,891968,Acetomicrobium mobile (strain ATCC BAA-54 / DS...,EBI,"23-JUL-2013 (Rel. 117, Last updated, Version 3)",131567;2;508458;649775;649776;649777;49894;97477
4,ACEP3,634452,Acetobacter pasteurianus (strain NBRC 3283 / L...,Genome Reviews,"24-NOV-2009 (Rel. 114, Last updated, Version 2)",131567;2;1224;28211;204441;433;434;438;481145


In [9]:
lside_names_a = set(species_df[species_df['Taxonomy_lineage'].\
                                          str.contains(';{};'.format(taxa_to_analyze[0]))]['OMA_code'])
lside_names_b = set(species_df[species_df['Taxonomy_lineage'].\
                                          str.contains(';{};'.format(taxa_to_analyze[1]))]['OMA_code'])

rside_names_a = set(species_df[species_df['Taxonomy_lineage'].\
                                          str.contains(';{};'.format(taxa_to_analyze[2]))]['OMA_code'])
rside_names_b = set(species_df[species_df['Taxonomy_lineage'].\
                                          str.contains(';{};'.format(taxa_to_analyze[3]))]['OMA_code'])
lside_names = lside_names_a.union(lside_names_b)
rside_names = rside_names_a.union(rside_names_b)
print(len(lside_names), len(rside_names))
print(len(lside_names_a) + len(lside_names_b), len(rside_names_a) + len(rside_names_b))

126 180
126 180


In [10]:
found_roots = []
for tree_file in glob.glob(direc_to_analyze + '*.treefile')[:]:
    tree = ete3.Tree(tree_file)
    rooting_methods_ete3.MP_root_ete3(tree)
    listy = is_monophyletic_ete3(tree, '', tree, set(lside_names), [])
#     listy = is_monophyletic_ete3(tree, '', tree, set(rside_names), [])
    found_roots.append(len(listy))
    if len(listy) == 1:
        tree.set_outgroup(listy[0][0])
        assert set(tree.children) == set(listy[0])
        
                
        if lside_names.union(set([i.name[:5] for i in tree.children[0].get_leaves()])) == lside_names:
            pass
        elif rside_names.union(set([i.name[:5] for i in tree.children[0].get_leaves()])) == rside_names:
            tree.swap_children()
        else:
            print('big error')
        assert lside_names.union(set([i.name[:5] for i in tree.children[0].get_leaves()])) == lside_names
        
        
        new_file = tree_file.replace('.afa.treefile', '.True.newick')
        label_internals(tree)
        tree.write(format=1, outfile=new_file)        
print('Successes (out of):', len([i for i in found_roots if i == 1]),  len(found_roots))
print('Possible errors:', len([i for i in found_roots if i > 1]))

Successes (out of): 1113 1154
Possible errors: 0


# Third dataset

In [11]:
direc_to_analyze = '../Data/OMA_orthologs/5204_4890_33511_33317_33090/'
taxa_to_analyze = direc_to_analyze.split('/')[-2].split('_')
print(taxa_to_analyze)
species_df = pd.read_csv('../Data/oma-species-full.tsv', sep='\t')
species_df.head()

['5204', '4890', '33511', '33317', '33090']


,OMA_code,TaxID,Scientific_name,Source,Release,Taxonomy_lineage
0,ACAM1,329726,Acaryochloris marina (strain MBIC 11017),Genome Reviews,"18-MAR-2008 (Rel. 88, Last updated, Version 2)",131567;2;1783272;1798711;1117;1890424;1890429;...
1,ACCPU,522306,Accumulibacter phosphatis (strain UW-1),EBI,"15-MAY-2014 (Rel. 120, Last updated, Version 4)",131567;2;1224;28216;119066;327159;327160
2,ACEAZ,574087,Acetohalobium arabaticum (strain ATCC 49924 / ...,EBI,"09-JAN-2014 (Rel. 119, Last updated, Version 5)",131567;2;1783272;1239;186801;53433;53434;28186...
3,ACEMN,891968,Acetomicrobium mobile (strain ATCC BAA-54 / DS...,EBI,"23-JUL-2013 (Rel. 117, Last updated, Version 3)",131567;2;508458;649775;649776;649777;49894;97477
4,ACEP3,634452,Acetobacter pasteurianus (strain NBRC 3283 / L...,Genome Reviews,"24-NOV-2009 (Rel. 114, Last updated, Version 2)",131567;2;1224;28211;204441;433;434;438;481145


In [12]:
lside_names_a = set(species_df[species_df['Taxonomy_lineage'].\
                                          str.contains(';{};'.format(taxa_to_analyze[0]))]['OMA_code'])
lside_names_b = set(species_df[species_df['Taxonomy_lineage'].\
                                          str.contains(';{};'.format(taxa_to_analyze[1]))]['OMA_code'])

lside_names_c = set(species_df[species_df['Taxonomy_lineage'].\
                                          str.contains(';{};'.format(taxa_to_analyze[2]))]['OMA_code'])
lside_names_d = set(species_df[species_df['Taxonomy_lineage'].\
                                          str.contains(';{};'.format(taxa_to_analyze[3]))]['OMA_code'])
rside_names = set(species_df[species_df['Taxonomy_lineage'].\
                                          str.contains(';{};'.format(taxa_to_analyze[4]))]['OMA_code'])

lside_names = lside_names_a.union(lside_names_b, lside_names_c, lside_names_d)
print(len(lside_names), len(rside_names))
print(len(lside_names_a) + len(lside_names_b) + len(lside_names_c) + len(lside_names_d), len(rside_names))

306 57
306 57


In [13]:
found_roots = []
for tree_file in glob.glob(direc_to_analyze + '*.treefile')[:]:
    tree = ete3.Tree(tree_file)
    rooting_methods_ete3.MP_root_ete3(tree)
    listy = is_monophyletic_ete3(tree, '', tree, set(lside_names), [])
#     listy = is_monophyletic_ete3(tree, '', tree, set(rside_names), [])
    found_roots.append(len(listy))
    if len(listy) == 1:
        tree.set_outgroup(listy[0][0])
        assert set(tree.children) == set(listy[0])
        
                
        if lside_names.union(set([i.name[:5] for i in tree.children[0].get_leaves()])) == lside_names:
            pass
        elif rside_names.union(set([i.name[:5] for i in tree.children[0].get_leaves()])) == rside_names:
            tree.swap_children()
        else:
            print('big error')
        assert lside_names.union(set([i.name[:5] for i in tree.children[0].get_leaves()])) == lside_names
        
        
        new_file = tree_file.replace('.afa.treefile', '.True.newick')
        label_internals(tree)
        tree.write(format=1, outfile=new_file)        
print('Successes (out of):', len([i for i in found_roots if i == 1]),  len(found_roots))
print('Possible errors:', len([i for i in found_roots if i > 1]))

Successes (out of): 893 944
Possible errors: 0
